In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

# 데이터 확인

In [491]:
df = pd.read_csv('Dubizzle_used_car_sales.csv')
df.head()

,title,price_in_aed,kilometers,body_condition,mechanical_condition,seller_type,body_type,no_of_cylinders,transmission_type,regional_specs,horsepower,fuel_type,steering_side,year,color,emirate,motors_trim,company,model,date_posted
0,MITSUBISHI PAJERO 3.5L / 2013,26000,167390,Perfect inside and out,Perfect inside and out,Dealer,SUV,6,Automatic Transmission,GCC Specs,Unknown,Gasoline,Left Hand Side,2013.0,Silver,Dubai,GLS,mitsubishi,pajero,13/05/2022
1,chevrolet silverado,110000,39000,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,North American Specs,400 - 500 HP,Gasoline,Left Hand Side,2018.0,White,Sharjah,1500 High Country,chevrolet,silverado,14/01/2022
2,MERCEDES-BENZ E300 - 2014 - GCC SPEC - FULL OP...,78000,200000,Perfect inside and out,Perfect inside and out,Dealer,Sedan,6,Automatic Transmission,GCC Specs,400 - 500 HP,Gasoline,Left Hand Side,2014.0,Blue,Sharjah,E 300,mercedes-benz,e-class,05/05/2022
3,WARRANTY UNTIL APR 2023 || Ferrari 488 Spider ...,899000,27000,Perfect inside and out,Perfect inside and out,Dealer,Hard Top Convertible,8,Automatic Transmission,GCC Specs,600 - 700 HP,Gasoline,Left Hand Side,2018.0,Red,Dubai,Standard,ferrari,488-spider,30/04/2022
4,USED RENAULT DOKKER 2020,33000,69000,Perfect inside and out,Perfect inside and out,Owner,Wagon,4,Manual Transmission,GCC Specs,Less than 150 HP,Gasoline,Left Hand Side,2020.0,White,Dubai,Standard,renault,dokker,13/05/2022


In [492]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9970 entries, 0 to 9969
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 9965 non-null   object 
 1   price_in_aed          9970 non-null   int64  
 2   kilometers            9970 non-null   int64  
 3   body_condition        9970 non-null   object 
 4   mechanical_condition  9970 non-null   object 
 5   seller_type           9970 non-null   object 
 6   body_type             9970 non-null   object 
 7   no_of_cylinders       9889 non-null   object 
 8   transmission_type     9970 non-null   object 
 9   regional_specs        9970 non-null   object 
 10  horsepower            9970 non-null   object 
 11  fuel_type             9970 non-null   object 
 12  steering_side         9970 non-null   object 
 13  year                  9000 non-null   float64
 14  color                 9970 non-null   object 
 15  emirate              

In [493]:
df.isnull().sum()

title                     5
price_in_aed              0
kilometers                0
body_condition            0
mechanical_condition      0
seller_type               0
body_type                 0
no_of_cylinders          81
transmission_type         0
regional_specs            0
horsepower                0
fuel_type                 0
steering_side             0
year                    970
color                     0
emirate                   0
motors_trim              28
company                   0
model                     0
date_posted               0
dtype: int64

# 데이터 처리

In [494]:
df["no_of_cylinders"].value_counts()

no_of_cylinders
6          3325
4          2993
8          2986
12          366
Unknown      57
3            56
10           53
5            53
Name: count, dtype: int64

In [495]:
# 1. Unknown을 결측치로 변환
df['no_of_cylinders'] = df['no_of_cylinders'].replace("Unknown", np.nan)

# 2. 문자열 -> 숫자형 변환
df['no_of_cylinders'] = df['no_of_cylinders'].astype(float)

# 3. 결측치 채우기 (최빈값 6)
df['no_of_cylinders'].fillna(6, inplace=True)

# 4. 다시 정수형 변환
df['no_of_cylinders'] = df['no_of_cylinders'].astype(int)

In [496]:
df["motors_trim"].value_counts()

motors_trim
Other              2175
Standard            432
SE                  232
S                   227
G 63 AMG            197
                   ... 
LTS                   1
SHO                   1
CLK 350               1
Limited Edition       1
SLC 200               1
Name: count, Length: 856, dtype: int64

In [497]:
df['motors_trim'].fillna('Unknown', inplace=True)  # 결측치만 처리

In [498]:
df["body_condition"].value_counts()

body_condition
Perfect inside and out                9612
No accidents, very few faults          208
A bit of wear & tear, all repaired     119
Normal wear & tear, a few issues        31
Name: count, dtype: int64

In [499]:
df['year'] = df['year'].fillna(df['year'].median())  # 결측치 중앙값 처리

In [500]:
df['car_age'] = 2025 - df['year']       # 차량 나이
df['is_perfect'] = (df['body_condition'] == 'Perfect inside and out').astype(int)    # 차량 상태를 이진화

df = df.drop(['body_condition', 'title'], axis=1)

'is_perfect'  컬럼으로 이진화 하였으니 'body_condition' 삭제

'title'은 예측과 관계가 없어 보이므로 삭제

In [501]:
df["mechanical_condition"].value_counts()

mechanical_condition
Perfect inside and out              9822
Minor faults, all fixed              136
Major faults fixed, small remain       8
Ongoing minor & major faults           3
Major faults, all fixed                1
Name: count, dtype: int64

In [502]:
# 두 개 범주로 단순화
df['is_mechanically_perfect'] = (df['mechanical_condition'] == 'Perfect inside and out').astype(int)
df = df.drop(columns=['mechanical_condition'])  # 단순화 해둔걸 사용하면 되므로 삭제

In [503]:
df["seller_type"].value_counts()

seller_type
Dealer                            6762
Owner                             2938
Dealership/Certified Pre-Owned     270
Name: count, dtype: int64

In [504]:
# 3개의 고르게 분포된 카테고리 -> 원핫인코딩
df = pd.get_dummies(df, columns=['seller_type'], drop_first=True)

In [505]:
df["body_type"].value_counts()

body_type
SUV                     4566
Sedan                   2748
Coupe                    851
Pick Up Truck            428
Hatchback                344
Sports Car               260
Hard Top Convertible     162
Van                      162
Soft Top Convertible     142
Crossover                114
Other                    102
Wagon                     66
Utility Truck             25
Name: count, dtype: int64

In [506]:
# 건수가 너무 적은 항목은 묶어서 Others로!

rare_body_types = df['body_type'].value_counts()[df['body_type'].value_counts() < 150].index
df['body_type'] = df['body_type'].replace(rare_body_types, 'Others')
df = pd.get_dummies(df, columns=['body_type'], drop_first=True)

In [507]:
df["transmission_type"].value_counts()

transmission_type
Automatic Transmission    9581
Manual Transmission        389
Name: count, dtype: int64

In [508]:
# 0과 1로 변환
df['is_automatic'] = (df['transmission_type'] == 'Automatic Transmission').astype(int)
df = df.drop(columns=['transmission_type'])

In [509]:
df["regional_specs"].value_counts()

regional_specs
GCC Specs               7601
North American Specs    1147
Other                    551
European Specs           422
Japanese Specs           249
Name: count, dtype: int64

In [510]:
# 원핫인코딩
df = pd.get_dummies(df, columns=['regional_specs'], drop_first=True)

In [511]:
df["horsepower"].value_counts()

horsepower
200 - 300 HP        2432
300 - 400 HP        2208
400 - 500 HP        1279
150 - 200 HP        1222
500 - 600 HP        1027
Unknown              814
Less than 150 HP     394
600 - 700 HP         391
700 - 800 HP         153
800 - 900 HP          30
900+ HP               20
Name: count, dtype: int64

In [512]:
# 수치형 범주로 인코딩
hp_order = {
    'Less than 150 HP': 1,
    '150 - 200 HP': 2,
    '200 - 300 HP': 3,
    '300 - 400 HP': 4,
    '400 - 500 HP': 5,
    '500 - 600 HP': 6,
    '600 - 700 HP': 7,
    '700 - 800 HP': 8,
    '800 - 900 HP': 9,
    '900+ HP': 10,
    'Unknown': np.nan
}

df['horsepower_encoded'] = df['horsepower'].map(hp_order)

# horsepower_encoded의 최빈값으로 대체
most_common_hp = df['horsepower_encoded'].mode()[0]
df['horsepower_encoded'].fillna(most_common_hp, inplace=True)

df = df.drop('horsepower', axis=1)  # 기존 컬럼 삭제

In [513]:
df["fuel_type"].value_counts()

fuel_type
Gasoline    9695
Diesel       167
Electric      85
Hybrid        23
Name: count, dtype: int64

In [514]:
# is_gasoline == 1 -> Gasoline
# is_gasoline == 0 -> Diesel, Electric, Hybrid 통합
df['is_gasoline'] = (df['fuel_type'] == 'Gasoline').astype(int)

df = df.drop('fuel_type', axis=1)

In [515]:
df["steering_side"].value_counts()

steering_side
Left Hand Side     9965
Right Hand Side       5
Name: count, dtype: int64

In [516]:
df = df.drop(columns=['steering_side'])

거의 모든 차가 좌측 운전석 -> 비교 의미 x

In [517]:
df["color"].value_counts()

color
White          3333
Black          2096
Grey           1325
Silver          851
Blue            712
Red             600
Gold            257
Brown           237
Green           142
Orange           79
Other Color      79
Yellow           78
Beige            73
Burgundy         71
Purple           24
Teal              8
Tan               5
Name: count, dtype: int64

In [518]:
# White, Black, Grey, Silver, Blue, Red는 유지 나머지는 Other Colors로 통합

rare_colors = df['color'].value_counts()[df['color'].value_counts() < 500].index
df['color'] = df['color'].replace(rare_colors, 'Other Colors')

In [519]:
df["model"].value_counts()

model
patrol               399
s-class              269
g-class              259
range-rover-sport    208
land-cruiser         200
                    ... 
wrangler-4xe           1
clk-class              1
dtsde-ville            1
polo                   1
odyssey-j              1
Name: count, Length: 552, dtype: int64

In [520]:
# 50개 미만 항목은 통합
rare_models = df['model'].value_counts()[df['model'].value_counts() < 50].index
df['model'] = df['model'].replace(rare_models, 'Other Models')

In [521]:
# date 파생변수 생성

df["date_posted"]=pd.to_datetime(df["date_posted"])
df["day_posted"]=(df["date_posted"]).dt.day
df["month_posted"]=(df["date_posted"]).dt.month
df["year_posted"]=(df["date_posted"]).dt.year
df['day_posted']=df['day_posted'].astype(int)
df['month_posted']=df['month_posted'].astype(int)
df['month_posted']=df['month_posted'].astype(int)

df = df.drop(["date_posted"], axis=1)

In [522]:
df.head()

,price_in_aed,kilometers,no_of_cylinders,year,color,emirate,motors_trim,company,model,car_age,...,is_automatic,regional_specs_GCC Specs,regional_specs_Japanese Specs,regional_specs_North American Specs,regional_specs_Other,horsepower_encoded,is_gasoline,day_posted,month_posted,year_posted
0,26000,167390,6,2013.0,Silver,Dubai,GLS,mitsubishi,pajero,12.0,...,1,True,False,False,False,3.0,1,13,5,2022
1,110000,39000,8,2018.0,White,Sharjah,1500 High Country,chevrolet,Other Models,7.0,...,1,False,False,True,False,5.0,1,14,1,2022
2,78000,200000,6,2014.0,Blue,Sharjah,E 300,mercedes-benz,e-class,11.0,...,1,True,False,False,False,5.0,1,5,5,2022
3,899000,27000,8,2018.0,Red,Dubai,Standard,ferrari,Other Models,7.0,...,1,True,False,False,False,7.0,1,30,4,2022
4,33000,69000,4,2020.0,White,Dubai,Standard,renault,Other Models,5.0,...,0,True,False,False,False,1.0,1,13,5,2022


In [523]:
numeric_df = df.select_dtypes(include=['int64', 'float64'])

# price_in_aed 기준으로 상관계수 절댓값 정렬
correlation_sorted = numeric_df.corr()["price_in_aed"].abs().sort_values(ascending=False)

print(correlation_sorted)

price_in_aed               1.000000
horsepower_encoded         0.553963
no_of_cylinders            0.475022
car_age                    0.205093
year                       0.205093
is_perfect                 0.070503
is_automatic               0.054014
is_mechanically_perfect    0.048426
day_posted                 0.031955
kilometers                 0.028398
month_posted               0.024237
is_gasoline                0.006749
Name: price_in_aed, dtype: float64


In [524]:
# 상위 1% 클리핑 -> 이상치 처리

price_cap = df['price_in_aed'].quantile(0.99)
df['price_in_aed_clipped'] = np.minimum(df['price_in_aed'], price_cap)

In [525]:
from sklearn.preprocessing import OrdinalEncoder

df['emirate'] = df['emirate'].str.strip()
one_hot_cols = ['color', 'emirate']
df_one_hot = pd.get_dummies(df[one_hot_cols], drop_first=True)

ordinal_cols = ['company', 'model', 'motors_trim']
ordinal_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_ordinal = pd.DataFrame(ordinal_enc.fit_transform(df[ordinal_cols]), columns=ordinal_cols)

numeric_and_binary = df.drop(columns=one_hot_cols + ordinal_cols + ['price_in_aed'])

df = pd.concat([numeric_and_binary, df_one_hot, df_ordinal], axis=1)

In [526]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9970 entries, 0 to 9969
Data columns (total 43 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   kilometers                                  9970 non-null   int64  
 1   no_of_cylinders                             9970 non-null   int64  
 2   year                                        9970 non-null   float64
 3   car_age                                     9970 non-null   float64
 4   is_perfect                                  9970 non-null   int64  
 5   is_mechanically_perfect                     9970 non-null   int64  
 6   seller_type_Dealership/Certified Pre-Owned  9970 non-null   bool   
 7   seller_type_Owner                           9970 non-null   bool   
 8   body_type_Hard Top Convertible              9970 non-null   bool   
 9   body_type_Hatchback                         9970 non-null   bool   
 10  body_type_Ot

# 모델 학습

In [527]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor

In [528]:
rf = RandomForestRegressor(random_state=42)
gbr = GradientBoostingRegressor(random_state=42)
xgb = XGBRegressor()

In [529]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

# 1. 데이터 정의
X = df.drop(columns=['price_in_aed_clipped'])
y = df['price_in_aed_clipped']

# 2. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 모델 정의
models = {
    "RandomForest": rf,
    "GradientBoosting": gbr,
    "XGBoost": xgb
}

# 4. 평가용 커스텀 RMSE 스코어러 (단위: AED)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

# 5. 결과 저장용 리스트
results = []

# 6. 학습 및 평가 루프
for name, model in models.items():
    print(f"\n모델: {name}")

    # 모델 학습
    model.fit(X_train, y_train)

    # 테스트 예측
    y_pred = model.predict(X_test)

    # 평가
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    test_r2 = r2_score(y_test, y_pred)

    # 교차검증
    cv_rmse_scores = -cross_val_score(model, X_train, y_train, cv=5, scoring=rmse_scorer)
    cv_r2_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

    print(f"Test RMSE : {test_rmse:.2f} AED")
    print(f"Test R2   : {test_r2:.4f}")
    print(f"CV RMSE   : {cv_rmse_scores.mean():.2f} AED")
    print(f"CV R2     : {cv_r2_scores.mean():.4f}")

    results.append({
        'Model': name,
        'Test_RMSE': test_rmse,
        'Test_R2': test_r2,
        'CV_RMSE': cv_rmse_scores.mean(),
        'CV_R2': cv_r2_scores.mean()
    })

# 7. 결과 정리
results_df = pd.DataFrame(results).sort_values('CV_RMSE')
results_df.reset_index(drop=True, inplace=True)
results_df


모델: RandomForest
Test RMSE : 113785.22 AED
Test R2   : 0.9143
CV RMSE   : 118026.90 AED
CV R2     : 0.9067

모델: GradientBoosting
Test RMSE : 128209.31 AED
Test R2   : 0.8912
CV RMSE   : 124888.78 AED
CV R2     : 0.8952

모델: XGBoost
Test RMSE : 106275.02 AED
Test R2   : 0.9253
CV RMSE   : 108258.07 AED
CV R2     : 0.9217


,Model,Test_RMSE,Test_R2,CV_RMSE,CV_R2
0,XGBoost,106275.020323,0.925253,108258.071567,0.921662
1,RandomForest,113785.220167,0.914315,118026.895747,0.906650
2,GradientBoosting,128209.312897,0.891214,124888.783857,0.895160


RandomForest와 XGBoost 선택

In [ ]:
from sklearn.inspection import permutation_importance

result_xgb = permutation_importance(xgb, X_test, y_test, scoring='r2', n_repeats=5, random_state=42)
perm_xgb = pd.Series(result_xgb.importances_mean, index=X_test.columns).sort_values(ascending=False)
top_xgb_perm = perm_xgb.index[:30].tolist()

result_rf = permutation_importance(rf, X_test, y_test, scoring='r2', n_repeats=5, random_state=42)
perm_rf = pd.Series(result_rf.importances_mean, index=X_test.columns).sort_values(ascending=False)
top_rf_perm = perm_rf.index[:30].tolist()

def evaluate_model(model, X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    test_r2 = r2_score(y_test, y_pred)

    rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
    cv_rmse = -cross_val_score(model, X_train, y_train, scoring=rmse_scorer, cv=5).mean()
    cv_r2 = cross_val_score(model, X_train, y_train, scoring='r2', cv=5).mean()

    return {
        'Model': name,
        'Test RMSE': test_rmse,
        'CV RMSE': cv_rmse,
        'Test R2': test_r2,
        'CV R2': cv_r2
    }

X = df.drop(columns=['price_in_aed_clipped'])
y = df['price_in_aed_clipped']

results = []

# XGBoost
results.append(evaluate_model(xgb, X[top_xgb_perm], y, "XGBoost-Top30"))

# RF
results.append(evaluate_model(rf, X[top_rf_perm], y, "RandomForest-Top30"))

results_df = pd.DataFrame(results)
results_df

,Model,Test RMSE,CV RMSE,Test R2,CV R2
0,XGBoost-PermTop30,102037.450367,106443.984691,0.931095,0.924276
1,RandomForest-PermTop30,113544.826982,117041.539912,0.914677,0.908268


xgboost와 랜덤포레스트 각각 피처 중요도를 기반으로 30개 학습!

xgboost 선택 -> 하이퍼파라미터 튜닝

In [538]:
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X[top_xgb_perm], y, test_size=0.2, random_state=42)

# 평가 지표
scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

# XGBoost
xgb_params = {
    'n_estimators': [300, 500],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 6],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_xgb = GridSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_grid=xgb_params,
    scoring=scorer,
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid_xgb.fit(X_train, y_train)


Fitting 5 folds for each of 32 candidates, totalling 160 fits


,estimator,"XGBRegressor(...bda=None, ...)"
,param_grid,"{'colsample_bytree': [0.8, 1.0], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 6], 'n_estimators': [300, 500], ...}"
,scoring,make_scorer(<...hod='predict')
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,objective,'reg:squarederror'


In [539]:
print("\nXGBoost Best RMSE:", -grid_xgb.best_score_)
print("Best Params:", grid_xgb.best_params_)
evaluate_model(grid_xgb.best_estimator_, X[top_xgb_perm], y, '그리드 서치')


XGBoost Best RMSE: 101324.73658249956
Best Params: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 500, 'subsample': 1.0}


{'Model': '그리드 서치',
 'Test RMSE': np.float64(101722.2581700732),
 'CV RMSE': np.float64(101324.73658249956),
 'Test R2': 0.9315198557698628,
 'CV R2': np.float64(0.9315832135991519)}